In [2]:
import requests
import os
import cx_Oracle
from bs4 import BeautifulSoup


conn = cx_Oracle.connect('camping', 'camping', 'localhost:1521/xe')
cursor = conn.cursor()

# SQL 구문
sql_str = "select * from camping_grounds_table"

# SQL 실행
cursor.execute(sql_str)

data = []

for i in range(1, 2):
    response = requests.get(f"https://www.gocamping.or.kr/bsite/camp/info/list.do?pageUnit=10&searchKrwd=&listOrdrTrget=c_rdcnt&searchDo=,9,&pageIndex={i}")
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.select(".camp_search_list")

    for item in items:
        for a_tag in item.select(".camp_tt a"):
            id = a_tag['href'].split('=')[1]
            site = "https://www.gocamping.or.kr/bsite/camp/info/read.do?c_no=" + id
            response2 = requests.get(site)
            html2 = response2.text
            soup2 = BeautifulSoup(html2, 'html.parser')

            # Get main image URL
            img = soup2.select_one(".img_b img")['src']
            img1 = "https://www.gocamping.or.kr" + img

            # Get other information
            name = soup2.select_one("[class='camp_name']").text
            address = soup2.find('th', text='주소').find_next_sibling('td').text.strip() if soup2.find('th', text='주소') else ""
            environment = soup2.find('th', text='캠핑장 환경').find_next_sibling('td').text.strip() if soup2.find('th', text='캠핑장 환경') else ""
            camping_type = soup2.find('th', text='캠핑장 유형').find_next_sibling('td').text.strip() if soup2.find('th', text='캠핑장 유형') else ""
            operating_period = soup2.find('th', text='운영기간').find_next_sibling('td').text.strip() if soup2.find('th', text='운영기간') else ""
            operating_days = soup2.find('th', text='운영일').find_next_sibling('td').text.strip() if soup2.find('th', text='운영일') else ""
            reservation_method = soup2.find('th', text='예약방법').find_next_sibling('td').text.strip() if soup2.find('th', text='예약방법') else ""
            nearby_facilities = soup2.find('th', text='주변이용가능시설').find_next_sibling('td').text.strip() if soup2.find('th', text='주변이용가능시설') else ""
            intro = soup2.select_one(".camp_intro_txt").text.strip() if soup2.select_one(".camp_intro_txt") else ""
            camp_items = [item.text.strip() for item in soup2.select(".camp_item_g span")]
            facility = soup2.find('th', text='주요시설').find_next_sibling('td').text.strip() if soup2.find('th', text='주요시설') else ""
            other_info = soup2.find('th', text='기타 정보').find_next_sibling('td').text.strip() if soup2.find('th', text='기타 정보') else ""
            additional_facility = soup2.find('th', text='기타 부대시설').find_next_sibling('td').text.strip() if soup2.find('th', text='기타 부대시설') else ""
            floor_type = soup2.find('th', text='바닥형태').find_next_sibling('td').text.strip() if soup2.find('th', text='바닥형태') else ""
            site_size = soup2.find('th', text='사이트 크기').find_next_sibling('td').text.strip() if soup2.find('th', text='사이트 크기') else ""
            camping_equipment_rental = soup2.find('th', text='캠핑장비대여').find_next_sibling('td').text.strip() if soup2.find('th', text='캠핑장비대여') else ""
            glamping_facilities = soup2.find('th', text='글램핑 내부시설').find_next_sibling('td').text.strip() if soup2.find('th', text='글램핑 내부시설') else ""
            caravan_facilities = soup2.find('th', text='카라반 내부시설').find_next_sibling('td').text.strip() if soup2.find('th', text='카라반 내부시설') else ""
            pet_access = soup2.find('th', text='반려동물 출입').find_next_sibling('td').text.strip() if soup2.find('th', text='반려동물 출입') else ""
            fireplace = soup2.find('th', text='화로대').find_next_sibling('td').text.strip() if soup2.find('th', text='화로대') else ""
            safety_facilities = soup2.find('th', text='안전시설현황').find_next_sibling('td').text.strip() if soup2.find('th', text='안전시설현황') else ""

            # Get Camp Intro Images
            img_urls_camp_intro = []
            img_elements_camp_intro = soup2.select(".camp_intro li img")
            for img_element in img_elements_camp_intro:
                img_url = "https://www.gocamping.or.kr" + img_element['src']  # Change relative URL to absolute URL
                img_urls_camp_intro.append(img_url)

            # Get Box Photo Images
            img_urls_box_photo = []
            img_elements_box_photo = soup2.select(".box_photo img")
            for img_element in img_elements_box_photo:
                img_url = img_element['src']
                img_urls_box_photo.append(img_url)

            # Append data to list
            data.append({
                "img": img1,
                "name": name,
                "address": address,
                "environment": environment,
                "camping_type": camping_type,
                "operating_period": operating_period,
                "operating_days": operating_days,
                "reservation_method": reservation_method,
                "nearby_facilities": nearby_facilities,
                "intro": intro,
                "camp_items": camp_items,
                "facility": facility,
                "other_info": other_info,
                "additional_facility": additional_facility,
                "floor_type": floor_type,
                "site_size": site_size,
                "camping_equipment_rental": camping_equipment_rental,
                "glamping_facilities": glamping_facilities,
                "caravan_facilities": caravan_facilities,
                "pet_access": pet_access,
                "fireplace": fireplace,
                "safety_facilities": safety_facilities,
                "camp_intro_images": img_urls_camp_intro,
                "box_photo_images": img_urls_box_photo
            })

# Print data
for camping in data:
    print("메인사진:", camping["img"])
    print("이름:", camping["name"])
    print("주소:", camping["address"])
    print("캠핑장 환경:", camping["environment"])
    print("캠핑장 유형:", camping["camping_type"])
    print("운영기간:", camping["operating_period"])
    print("운영일:", camping["operating_days"])
    print("예약방법:", camping["reservation_method"])
    print("주변이용가능시설:", camping["nearby_facilities"])
    print("캠핑장 소개:", camping["intro"])
    print("캠핑장 시설:", camping["camp_items"])
    print("주요시설:", camping["facility"])
    print("기타 정보:", camping["other_info"])
    print("기타 부대시설:", camping["additional_facility"])
    print("바닥형태:", camping["floor_type"])
    print("사이트 크기:", camping["site_size"])
    print("캠핑장비대여:", camping["camping_equipment_rental"])
    print("글램핑 내부시설:", camping["glamping_facilities"])
    print("카라반 내부시설:", camping["caravan_facilities"])
    print("반려동물 출입:", camping["pet_access"])
    print("화로대:", camping["fireplace"])
    print("안전시설현황:", camping["safety_facilities"])
    print("소개사진:", camping["camp_intro_images"])
    print("캠핑장사진:", camping["box_photo_images"])

    # Insert data into the database
    cursor.execute("INSERT INTO camping_grounds_table (img_url, name, address, environment, camping_type, operating_period, operating_days, reservation_method, nearby_facilities, intro, camp_items, facility, other_info, additional_facility, floor_type, site_size, camping_equipment_rental, glamping_facilities, caravan_facilities, pet_access, fireplace, safety_facilities, camp_intro_images, box_photo_images) VALUES (:img, :name, :address, :environment, :camping_type, :operating_period, :operating_days, :reservation_method, :nearby_facilities, :intro, :camp_items, :facility, :other_info, :additional_facility, :floor_type, :site_size, :camping_equipment_rental, :glamping_facilities, :caravan_facilities, :pet_access, :fireplace, :safety_facilities, :camp_intro_images, :box_photo_images)",
                   img=camping["img"],
                   name=camping["name"],
                   address=camping["address"],
                   environment=camping["environment"],
                   camping_type=camping["camping_type"],
                   operating_period=camping["operating_period"],
                   operating_days=camping["operating_days"],
                   reservation_method=camping["reservation_method"],
                   nearby_facilities=camping["nearby_facilities"],
                   intro=camping["intro"],
                   camp_items=', '.join(camping["camp_items"]),
                   facility=camping["facility"],
                   other_info=camping["other_info"],
                   additional_facility=camping["additional_facility"],
                   floor_type=camping["floor_type"],
                   site_size=camping["site_size"],
                   camping_equipment_rental=camping["camping_equipment_rental"],
                   glamping_facilities=camping["glamping_facilities"],
                   caravan_facilities=camping["caravan_facilities"],
                   pet_access=camping["pet_access"],
                   fireplace=camping["fireplace"],
                   safety_facilities=camping["safety_facilities"],
                   camp_intro_images=', '.join(camping["camp_intro_images"]),
                   box_photo_images=', '.join(camping["box_photo_images"])
                   )

# 커밋하여 변경사항을 저장
conn.commit()

cursor.close()
conn.close()

C:\Users\Admin\AppData\Local\Temp\ipykernel_10588\2192349919.py:38: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  address = soup2.find('th', text='주소').find_next_sibling('td').text.strip() if soup2.find('th', text='주소') else ""
C:\Users\Admin\AppData\Local\Temp\ipykernel_10588\2192349919.py:39: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  environment = soup2.find('th', text='캠핑장 환경').find_next_sibling('td').text.strip() if soup2.find('th', text='캠핑장 환경') else ""
C:\Users\Admin\AppData\Local\Temp\ipykernel_10588\2192349919.py:40: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  camping_type = soup2.find('th', text='캠핑장 유형').find_next_sibling('td').text.strip() if soup2.find('th', text='캠핑장 유형') else ""
C:\Users\Admin\AppData\Local\Temp\ipykernel_10588\2192349919.py:41: DeprecationWarning: The 'text' argument to find()-type 

메인사진: https://www.gocamping.or.kr/upload/camp/2422/thumb/thumb_720_1819fKOADYFML8JkBiVZiTKc.jpg
이름: 
						용인 자연숲 캠핑장
주소: 경기 용인시 처인구 원삼면 원양로591번길 93-34
캠핑장 환경: 산 / 민간
캠핑장 유형: 일반야영장, 자동차야영장
운영기간: 봄, 여름, 가을, 겨울
운영일: 평일+주말
예약방법: 전화
주변이용가능시설: 계곡 물놀이, 산책로, 낚시, 해수욕, 수영장, 청소년체험시설
캠핑장 소개: 용인자연숲힐링캠핑장은 양지 나들목 가까이에 있다. 지도상에서 보면 양지파인리조트 산 너머에 있다. 좌항 저수지 위쪽에 숲속에 있는 자연친화적 전망좋은 캠핑장으로 아이들을 위한 대형 어린이 수영장, 방방이, 모레놀이터와 같은 놀이시설이 잘 갖춰져 있다.  캠핑장 위로 한옥펜션도 있어 분위기가 더욱 좋다. 최종 정보 수정일 : 2018-09-21
캠핑장 시설: ['전기', '와이파이', '장작판매', '온수', '트렘폴린', '물놀이장', '운동시설']
주요시설: 자동차야영장사이트(38면)
기타 정보: 개인 트레일러 입장가능
기타 부대시설: 
바닥형태: 
사이트 크기: 
캠핑장비대여: 텐트
식기
침낭
글램핑 내부시설: 
카라반 내부시설: 
반려동물 출입: 불가능
화로대: 개별
안전시설현황: 소화기 (10)
방화수 (10)
화재감지기 (3)
소개사진: ['https://www.gocamping.or.kr/upload/camp/2422/thumb/thumb_384_4941NqXY3hlgnreJh1OB5lVA.jpg', 'https://www.gocamping.or.kr/upload/camp/2422/thumb/thumb_384_5011uE9cKpVPtojiu2GElTaY.jpg', 'https://www.gocamping.or.kr/upload/camp/2422/thumb/thumb_384_9792p9i6yF8HAmamUqk0Fjcr.jpg']
캠핑장사진: ['https: